In [ ]:
import pandas as pd
import requests
import csv
import time

API_KEY = 'Your_API_KEY_here'  # Replace with your Google Places API key
TEXT_SEARCH_URL = "https://maps.googleapis.com/maps/api/place/textsearch/json"
DETAILS_URL = "https://maps.googleapis.com/maps/api/place/details/json"

def get_place_ids(search_term):
    params = {
        'query': search_term,
        'key': API_KEY,
    }

    place_ids = []

    while True:
        response = requests.get(TEXT_SEARCH_URL, params=params)
        if response.status_code == 200:
            results = response.json()
            place_ids.extend([result['place_id'] for result in results['results']])
            if 'next_page_token' not in results:
                break
            else:
                params['pagetoken'] = results['next_page_token']
        else:
            print(f"Error: {response.status_code}")
            print(f"Message: {response.json()}")
            break
        # Ensure enough delay before next request
        time.sleep(2)

    return place_ids

def get_place_details(place_id):
    params = {
        'place_id': place_id,
        'fields': 'name,types,geometry/location,formatted_phone_number',
        'key': API_KEY,
    }

    response = requests.get(DETAILS_URL, params=params)

    if response.status_code == 200:
        return response.json()['result']
    else:
        print(f"Error: {response.status_code}")
        print(f"Message: {response.json()}")
        return None

def create_csv(data, filename):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Name', 'Types', 'Latitude', 'Longitude', 'Phone Number'])

        for place in data:
            writer.writerow([place['name'], ", ".join(place['types']), place['geometry']['location']['lat'], place['geometry']['location']['lng'], place.get('formatted_phone_number', 'N/A')])

if __name__ == "__main__":
    search_term = input("Enter a search term: ")
    place_ids = get_place_ids(search_term)
    
    if place_ids is not None:
        place_details = [get_place_details(place_id) for place_id in place_ids]
        create_csv(place_details, 'google_places.csv')
        print("CSV file has been created!")
    else:
        print("Failed to get place data!")
